In [31]:
import pandas as pd
import time #to have today's date
timestr = time.strftime("%Y%m%d")

In [3]:
#get the LDA model from another notebook
%run ./p_07_final_LDA_paper.ipynb

#doc_term_matrix_x 
#ldamodel =
#var_drop_x #drop all documents with number of words <= x. Only keep documents >x
#doc_clean_x 
#doc_list_paras_x

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

Bad key text.latex.preview in file c:\users\mann\appdata\local\programs\python\python37-32\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file c:\users\mann\appdata\local\programs\python\python37-32\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplot

## do dataframe with all the topics+categories+keywords and the text+questions per document

### create the dataframe

In [21]:
#create dataframe with empty columns
topics_per_doc_df = pd.DataFrame()
list_topics_columns = list() #to use later
for x in range(0,var_no_topics):
    topics_per_doc_df["perc_cont_"+str(x)] = []
    topics_per_doc_df["keywords_"+str(x)] = []
    list_topics_columns.append("perc_cont_"+str(x))

In [22]:
# Get relevant topics for each document, this takes a while
for i, row in enumerate(ldamodel[doc_term_matrix_x]): #this provides a list with all topics and their percentage contribution
    # Get all the topics, Perc Contribution and Keywords for each document
    for j in row:
        topic_num = j[0]
        prop_topic = j[1]
        wp = ldamodel.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        topics_per_doc_df.loc[i, "perc_cont_"+str(topic_num)] = prop_topic
        topics_per_doc_df.loc[i, "keywords_"+str(topic_num)] = topic_keywords

KeyboardInterrupt: 

In [ ]:
# Add original text to the end of the output
contents = pd.Series(doc_list_paras_x)
topics_per_doc_df = pd.concat([topics_per_doc_df, contents], axis=1)
topics_per_doc_df.rename(columns={0:'Text'}, inplace=True) #rename the added column with the original text

In [ ]:
#no need anymore, question is already in text
#if available, also add the question that was asked to each document
#df_questions_replies = pd.read_csv('.//results//topics_group//questions_replies_'+timestr+'_V01.csv',
#    sep=';', decimal=',', index_col=0)
df_docs_topics = topics_per_doc_df#.merge(df_questions_replies)

In [8]:
#merge to get information on country, class etc. for each document
df_docs_topics_country_class = df_all_interviews.merge(df_docs_topics) #do I need df_all_interviews_x? -->no!
#P: the text "I don't have information on that." is said two times.
#thus, when we merge the dataframe, it appears four times. 
#S: drop duplicate rows:
df_docs_topics_country_class = df_docs_topics_country_class.drop_duplicates().\
        reset_index(drop=True)

In [9]:
df_docs_topics_country_class.to_csv('.//results//master_dfs//docs_oldNum_topics_country_class_keywords_text_q_'+timestr+'_V01.csv',
    sep=';', decimal=',')

## change numbers to new topic numbers

In [23]:
#get dataframe with the categories (numbers & names), topics (numbers (ori&new) & names), and keywords and lambda
df_topics_categories = pd.read_csv\
('.//results//master_dfs//category_numname_topics_numname_orinew_keywords_lambda_'+timestr+'_V01.csv',
    sep=';', decimal=',', index_col=0)

In [24]:
df_topic_nums = df_topics_categories[['ori_topic_num', 'Topic_num']].sort_values('ori_topic_num')
#the keys are the old (original) topic numbers, the value are the new ones.
dict_t_nums = dict(zip(df_topic_nums.ori_topic_num, df_topic_nums.Topic_num))
#so for all the keys (for x in dict_t_nums), if x is the number, change x to dict_t_nums[x]

In [25]:
#change the numbers
df_docs_newNum_topics_country_class = df_docs_topics_country_class.copy()

for x in df_docs_newNum_topics_country_class.columns:
    list_x_split = x.split('_')
    if list_x_split[0] == 'perc': #so column is 'perc_cont_x'
        var_number = int(list_x_split[2])
        df_docs_newNum_topics_country_class.rename(columns = {x:'t_'+str(dict_t_nums[var_number])+'_perc_cont'}, inplace = True)
    if list_x_split[0] == 'keywords': #so column is 'perc_cont_x'
        var_number = int(list_x_split[1])
        df_docs_newNum_topics_country_class.rename(columns = {x:'t_'+str(dict_t_nums[var_number])+'_keywords'}, inplace = True)

In [26]:
#rearrange column names:
list_perc = []
for x in range(1, df_topic_nums['ori_topic_num'].max()+2):
    list_perc.append('t_'+str(x)+'_perc_cont')
    list_perc.append('t_'+str(x)+'_keywords')

In [27]:
list_columns = ['Country', 'no_interview', 'class', 'Text'] + list_perc #+ ['Question']

In [28]:
df_docs_newNum_topics_country_class_order = df_docs_newNum_topics_country_class[list_columns]

In [29]:
df_docs_newNum_topics_country_class_order.tail(3)

,Country,no_interview,class,Text,t_1_perc_cont,t_1_keywords,t_2_perc_cont,t_2_keywords,t_3_perc_cont,t_3_keywords,...,t_18_perc_cont,t_18_keywords,t_19_perc_cont,t_19_keywords,t_20_perc_cont,t_20_keywords,t_21_perc_cont,t_21_keywords,t_22_perc_cont,t_22_keywords
4983,VN,11,socint,<q/start> do you think that for example the c...,0.136787,"people, work, political, talk, time, community...",0.000221,"everyones, metro, insurance, expenditure, char...",0.000221,"job, labor, worker, trade_union, secretary, em...",...,0.131321,"emission, climate_change, target, sector, carb...",0.000221,"net_zero, decarbonisation, geothermal, health,...",0.000221,"uk, year, gas, big, government, around, renewa...",0.000221,"challenge, goal, development, important, persp...",0.037822,"coal, important, phaseout, question, policy, r..."
4984,VN,11,socint,"<q/start> thank you so much, that was very in...",0.000657,"people, work, political, talk, time, community...",0.000657,"everyones, metro, insurance, expenditure, char...",0.000657,"job, labor, worker, trade_union, secretary, em...",...,0.000657,"emission, climate_change, target, sector, carb...",0.000657,"net_zero, decarbonisation, geothermal, health,...",0.000657,"uk, year, gas, big, government, around, renewa...",0.000657,"challenge, goal, development, important, persp...",0.000657,"coal, important, phaseout, question, policy, r..."
4985,VN,11,socint,<q/start> yes because now I mean when you loo...,0.068875,"people, work, political, talk, time, community...",0.000158,"everyones, metro, insurance, expenditure, char...",0.000158,"job, labor, worker, trade_union, secretary, em...",...,0.000158,"emission, climate_change, target, sector, carb...",0.000158,"net_zero, decarbonisation, geothermal, health,...",0.000158,"uk, year, gas, big, government, around, renewa...",0.000158,"challenge, goal, development, important, persp...",0.000158,"coal, important, phaseout, question, policy, r..."


## export dataframe

In [32]:
df_docs_newNum_topics_country_class_order.to_csv('.//results//master_dfs//docs_newNum_topics_country_class_keywords_text_q_'+timestr+'_V01.csv',
    sep=';', decimal=',')